In [1]:
import sys

sys.path.append("../")
import os
import math
import numpy as np
import cv2
import pandas as pd
import tikzplotlib
from torch.utils.data import DataLoader, ConcatDataset
from pedrec.models.constants.action_mappings import ACTION
from pedrec.configs.dataset_configs import get_sim_dataset_cfg_default
from pedrec.datasets.pedrec_dataset import PedRecDataset
from pedrec.configs.pedrec_net_config import PedRecNet50Config
from pedrec.models.constants.dataset_constants import DatasetType
from pedrec.visualizers.skeleton_visualizer import draw_skeleton
from pedrec.visualizers.skeleton_3d_visualizer import add_skeleton_3d_to_axes
from pedrec.visualizers.visualization_helper_3d import draw_grid_3d, draw_origin_3d
from random import randint
import matplotlib.pyplot as plt
%matplotlib widget


In [2]:
cfg = PedRecNet50Config()

# ROM Train
dataset_cfg = get_sim_dataset_cfg_default()
dataset_root = "data/datasets/Conti01/"
dataset_df_train_filename = "rt_conti_01_train_FIN.pkl"
dataset_df_val_filename = "rt_conti_01_val_FIN.pkl"

train_dataset = PedRecDataset(dataset_root, dataset_df_train_filename, DatasetType.VALIDATE, dataset_cfg, cfg.model.input_size, None)
val_dataset = PedRecDataset(dataset_root, dataset_df_val_filename, DatasetType.VALIDATE, dataset_cfg, cfg.model.input_size, None)

dataset = ConcatDataset([train_dataset, val_dataset])
dataset_length = len(dataset)
print(len(train_dataset))
print(train_dataset.info.full_length)
print(train_dataset.info.used_length)
print(len(val_dataset))
print(val_dataset.info.full_length)
print(val_dataset.info.used_length)
# dataset = val_dataset
# dataset_length = len(dataset)

1166130
1347056
1166130
121859
140984
121859


In [3]:
# dataset_df_val_path = os.path.join(dataset_root, dataset_df_val_filename)
# df_val = pd.read_pickle(dataset_df_val_path)
# pd.options.display.max_columns = None
# pd.options.display.float_format= '{:.2f}'.format
# skeleton2d_visibles = [col for col in df_val if col.startswith('skeleton2d') and col.endswith('_visible')]
# df_val["visible_joints"] = df_val[skeleton2d_visibles].sum(axis=1)
# df_val[df_val["visible_joints"] < 3].head(5)

In [3]:
fig, ax = plt.subplots(3,3, figsize=(10,10))
fig_3d = plt.figure(figsize=(10,10))

count = 0
for i in range(0, 3):
    for j in range(0, 3):
        index = randint(0, len(val_dataset))
        entry = val_dataset[index]
        # entry = dataset[count + 606]
        model_input, labels = entry
        skeleton = labels["skeleton"]
        skeleton_3d = labels["skeleton_3d"]
        scale_factor = 3
        skeleton_3d[:, :3] *= scale_factor
        skeleton_3d[:, :3] -= (scale_factor / 2)
        center = labels["center"]
        scale = labels["scale"]
        rotation = labels["rotation"]
        is_real_img = labels["is_real_img"]
        img_path = labels["img_path"]
        skeleton[:, 0] *= model_input.shape[1]
        skeleton[:, 1] *= model_input.shape[0]
        visible_joints = np.sum(skeleton[:, 2])
        body_orientation = labels["orientation"][0][1]
        body_orientation *= 2*math.pi
        body_orientation = math.degrees(body_orientation)
        head_orientation = labels["orientation"][1][1]
        head_orientation *= 2*math.pi
        head_orientation = math.degrees(head_orientation)
        draw_skeleton(model_input, skeleton)
        img = model_input
        ax[i, j].imshow(img)
        ax[i, j].set_title(f"{index}: {count}: {visible_joints} | Bθ​: {body_orientation:.1f}° | Hθ​: {head_orientation:.1f}°")
        ax[i, j].set_title(f"{index}")
        ax_3d = fig_3d.add_subplot(3, 3, count+1, projection='3d')
        draw_grid_3d(ax_3d, lim=1)
        draw_origin_3d(ax_3d)
        add_skeleton_3d_to_axes(ax_3d, skeleton_3d, size=4)
        ax_3d.set_title(f"{index}: {count}: {visible_joints} | Bθ​: {body_orientation:.1f}° | Hθ​: {head_orientation:.1f}°")

        # print(f"{count}: Model input shape: {model_input.shape}, Min value: {model_input.min()}, max value: {model_input.max()}")
        # print(f"{count}: center: {center}, scale: {scale}, rotation: {rotation}")
        # print(f"{count}: is_real_img: {is_real_img}")
        # print(f"{count}: visible joints: {visible_joints}")
        # print(f"{count}: path: {img_path}")
        # print("------------")
        count += 1
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
dataset_df_train_path = os.path.join(dataset_root, dataset_df_train_filename)
df_train = pd.read_pickle(dataset_df_train_path)
dataset_df_val_path = os.path.join(dataset_root, dataset_df_val_filename)
df_val = pd.read_pickle(dataset_df_val_path)
df = pd.concat([df_train, df_val], ignore_index=True)
pd.options.display.max_columns = None
pd.options.display.float_format= '{:.2f}'.format
print(f"Number of entrys: {df.shape[0]}")

Number of entrys: 1488040


In [4]:
filter_skeleton2d = [col for col in df if col.startswith('skeleton2d')]
filter_skeleton3d = [col for col in df if col.startswith('skeleton3d')]
filter_bb = [col for col in df if col.startswith('bb')]
filter_body_orientation = [col for col in df if col.startswith('body_orientation')]
filter_head_orientation = [col for col in df if col.startswith('head_orientation')]
filter_env = [col for col in df if col.startswith('env')]

In [5]:
# skeleton 2d calculated columns
skeleton2d_xs = [col for col in df if col.startswith('skeleton2d') and col.endswith('_x')]
skeleton2d_ys = [col for col in df if col.startswith('skeleton2d') and col.endswith('_y')]
skeleton2d_visibles = [col for col in df if col.startswith('skeleton2d') and col.endswith('_visible')]
df["skeleton2d_width"] = df[skeleton2d_xs].max(axis=1) - df[skeleton2d_xs].min(axis=1)
df["skeleton2d_height"] = df[skeleton2d_ys].max(axis=1) - df[skeleton2d_ys].min(axis=1)
df["skeleton2d_size"] = np.sqrt(df["skeleton2d_width"]**2 + df["skeleton2d_height"]**2)
df["skeleton2d_visible_joints"] = df[skeleton2d_visibles].sum(axis=1)

df_train["skeleton2d_width"] = df_train[skeleton2d_xs].max(axis=1) - df_train[skeleton2d_xs].min(axis=1)
df_train["skeleton2d_height"] = df_train[skeleton2d_ys].max(axis=1) - df_train[skeleton2d_ys].min(axis=1)
df_train["skeleton2d_size"] = np.sqrt(df_train["skeleton2d_width"]**2 + df_train["skeleton2d_height"]**2)
df_train["skeleton2d_visible_joints"] = df_train[skeleton2d_visibles].sum(axis=1)

df_val["skeleton2d_width"] = df_val[skeleton2d_xs].max(axis=1) - df_val[skeleton2d_xs].min(axis=1)
df_val["skeleton2d_height"] = df_val[skeleton2d_ys].max(axis=1) - df_val[skeleton2d_ys].min(axis=1)
df_val["skeleton2d_size"] = np.sqrt(df_val["skeleton2d_width"]**2 + df_val["skeleton2d_height"]**2)
df_val["skeleton2d_visible_joints"] = df_val[skeleton2d_visibles].sum(axis=1)

In [6]:
# skeleton 3d calculated columns
skeleton3d_xs = [col for col in df if col.startswith('skeleton3d') and col.endswith('_x')]
skeleton3d_ys = [col for col in df if col.startswith('skeleton3d') and col.endswith('_y')]
skeleton3d_zs = [col for col in df if col.startswith('skeleton3d') and col.endswith('_z')]
df["skeleton3d_width"] = df[skeleton3d_xs].max(axis=1) - df[skeleton3d_xs].min(axis=1)
df["skeleton3d_height"] = df[skeleton3d_ys].max(axis=1) - df[skeleton3d_ys].min(axis=1)
df["skeleton3d_depth"] = df[skeleton3d_zs].max(axis=1) - df[skeleton3d_zs].min(axis=1)
df["skeleton3d_size"] = np.sqrt(df["skeleton3d_width"]**2 + df["skeleton3d_height"]**2 + df["skeleton3d_depth"]**2)

df_train["skeleton3d_width"] = df_train[skeleton3d_xs].max(axis=1) - df_train[skeleton3d_xs].min(axis=1)
df_train["skeleton3d_height"] = df_train[skeleton3d_ys].max(axis=1) - df_train[skeleton3d_ys].min(axis=1)
df_train["skeleton3d_depth"] = df_train[skeleton3d_zs].max(axis=1) - df_train[skeleton3d_zs].min(axis=1)
df_train["skeleton3d_size"] = np.sqrt(df_train["skeleton3d_width"]**2 + df_train["skeleton3d_height"]**2 + df_train["skeleton3d_depth"]**2)

df_val["skeleton3d_width"] = df_val[skeleton3d_xs].max(axis=1) - df_val[skeleton3d_xs].min(axis=1)
df_val["skeleton3d_height"] = df_val[skeleton3d_ys].max(axis=1) - df_val[skeleton3d_ys].min(axis=1)
df_val["skeleton3d_depth"] = df_val[skeleton3d_zs].max(axis=1) - df_val[skeleton3d_zs].min(axis=1)
df_val["skeleton3d_size"] = np.sqrt(df_val["skeleton3d_width"]**2 + df_val["skeleton3d_height"]**2 + df_val["skeleton3d_depth"]**2)

In [7]:
# remove empty frames
df = df[df["skeleton2d_size"] > 5]
df = df[df["skeleton3d_size"] > 10]

df_train = df_train[df_train["skeleton2d_size"] > 5]
df_train = df_train[df_train["skeleton3d_size"] > 10]

df_val = df_val[df_val["skeleton2d_size"] > 5]
df_val = df_val[df_val["skeleton3d_size"] > 10]

In [8]:
# distance calculated column
env_position_xs = [col for col in df if col == "env_position_x"] 
env_position_ys = [col for col in df if col == "env_position_y"] 
env_position_zs = [col for col in df if col == "env_position_z"] 

df["distance_xz"] = np.sqrt(np.abs(df[env_position_xs].sum(axis=1)**2) + np.abs(df[env_position_zs].sum(axis=1)**2))
df_train["distance_xz"] = np.sqrt(np.abs(df_train[env_position_xs].sum(axis=1)**2) + np.abs(df_train[env_position_zs].sum(axis=1)**2))
df_val["distance_xz"] = np.sqrt(np.abs(df_val[env_position_xs].sum(axis=1)**2) + np.abs(df_val[env_position_zs].sum(axis=1)**2))

df["body_orientation_phi"] = df["body_orientation_phi"] * math.pi * 2
df["body_orientation_theta"] = df["body_orientation_theta"] * math.pi
df["head_orientation_phi"] = df["head_orientation_phi"] * math.pi * 2
df["head_orientation_theta"] = df["head_orientation_theta"] * math.pi

df_val["body_orientation_phi"] = df_val["body_orientation_phi"] * math.pi * 2
df_val["body_orientation_theta"] = df_val["body_orientation_theta"] * math.pi
df_val["head_orientation_phi"] = df_val["head_orientation_phi"] * math.pi * 2
df_val["head_orientation_theta"] = df_val["head_orientation_theta"] * math.pi

df_train["body_orientation_phi"] = df_train["body_orientation_phi"] * math.pi * 2
df_train["body_orientation_theta"] = df_train["body_orientation_theta"] * math.pi
df_train["head_orientation_phi"] = df_train["head_orientation_phi"] * math.pi * 2
df_train["head_orientation_theta"] = df_train["head_orientation_theta"] * math.pi

In [10]:
df.head(5)

,dataset,dataset_type,scene_id,scene_start,scene_end,frame_nr_global,frame_nr_local,img_dir,img_id,img_type,subject_id,gender,skin_color,size,bmi,age,movement,movement_speed,is_real_img,actions,bb_center_x,bb_center_y,bb_width,bb_height,bb_score,bb_class,env_position_x,env_position_y,env_position_z,body_orientation_theta,body_orientation_phi,body_orientation_score,body_orientation_visible,head_orientation_theta,head_orientation_phi,head_orientation_score,head_orientation_visible,skeleton2d_nose_x,skeleton2d_nose_y,skeleton2d_nose_score,skeleton2d_nose_visible,skeleton2d_nose_supported,skeleton2d_left_eye_x,skeleton2d_left_eye_y,skeleton2d_left_eye_score,skeleton2d_left_eye_visible,skeleton2d_left_eye_supported,skeleton2d_right_eye_x,skeleton2d_right_eye_y,skeleton2d_right_eye_score,skeleton2d_right_eye_visible,skeleton2d_right_eye_supported,skeleton2d_left_ear_x,skeleton2d_left_ear_y,skeleton2d_left_ear_score,skeleton2d_left_ear_visible,skeleton2d_left_ear_supported,skeleton2d_right_ear_x,skeleton2d_right_ear_y,skeleton2d_right_ear_score,skeleton2d_right_ear_visible,skeleton2d_right_ear_supported,skeleton2d_left_shoulder_x,skeleton2d_left_shoulder_y,skeleton2d_left_shoulder_score,skeleton2d_left_shoulder_visible,skeleton2d_left_shoulder_supported,skeleton2d_right_shoulder_x,skeleton2d_right_shoulder_y,skeleton2d_right_shoulder_score,skeleton2d_right_shoulder_visible,skeleton2d_right_shoulder_supported,skeleton2d_left_elbow_x,skeleton2d_left_elbow_y,skeleton2d_left_elbow_score,skeleton2d_left_elbow_visible,skeleton2d_left_elbow_supported,skeleton2d_right_elbow_x,skeleton2d_right_elbow_y,skeleton2d_right_elbow_score,skeleton2d_right_elbow_visible,skeleton2d_right_elbow_supported,skeleton2d_left_wrist_x,skeleton2d_left_wrist_y,skeleton2d_left_wrist_score,skeleton2d_left_wrist_visible,skeleton2d_left_wrist_supported,skeleton2d_right_wrist_x,skeleton2d_right_wrist_y,skeleton2d_right_wrist_score,skeleton2d_right_wrist_visible,skeleton2d_right_wrist_supported,skeleton2d_left_hip_x,skeleton2d_left_hip_y,skeleton2d_left_hip_score,skeleton2d_left_hip_visible,skeleton2d_left_hip_supported,skeleton2d_right_hip_x,skeleton2d_right_hip_y,skeleton2d_right_hip_score,skeleton2d_right_hip_visible,skeleton2d_right_hip_supported,skeleton2d_left_knee_x,skeleton2d_left_knee_y,skeleton2d_left_knee_score,skeleton2d_left_knee_visible,skeleton2d_left_knee_supported,skeleton2d_right_knee_x,skeleton2d_right_knee_y,skeleton2d_right_knee_score,skeleton2d_right_knee_visible,skeleton2d_right_knee_supported,skeleton2d_left_ankle_x,skeleton2d_left_ankle_y,skeleton2d_left_ankle_score,skeleton2d_left_ankle_visible,skeleton2d_left_ankle_supported,skeleton2d_right_ankle_x,skeleton2d_right_ankle_y,skeleton2d_right_ankle_score,skeleton2d_right_ankle_visible,skeleton2d_right_ankle_supported,skeleton2d_hip_center_x,skeleton2d_hip_center_y,skeleton2d_hip_center_score,skeleton2d_hip_center_visible,skeleton2d_hip_center_supported,skeleton2d_spine_center_x,skeleton2d_spine_center_y,skeleton2d_spine_center_score,skeleton2d_spine_center_visible,skeleton2d_spine_center_supported,skeleton2d_neck_x,skeleton2d_neck_y,skeleton2d_neck_score,skeleton2d_neck_visible,skeleton2d_neck_supported,skeleton2d_head_lower_x,skeleton2d_head_lower_y,skeleton2d_head_lower_score,skeleton2d_head_lower_visible,skeleton2d_head_lower_supported,skeleton2d_head_upper_x,skeleton2d_head_upper_y,skeleton2d_head_upper_score,skeleton2d_head_upper_visible,skeleton2d_head_upper_supported,skeleton2d_left_foot_end_x,skeleton2d_left_foot_end_y,skeleton2d_left_foot_end_score,skeleton2d_left_foot_end_visible,skeleton2d_left_foot_end_supported,skeleton2d_right_foot_end_x,skeleton2d_right_foot_end_y,skeleton2d_right_foot_end_score,skeleton2d_right_foot_end_visible,skeleton2d_right_foot_end_supported,skeleton2d_left_hand_end_x,skeleton2d_left_hand_end_y,skeleton2d_left_hand_end_score,skeleton2d_left_hand_end_visible,skeleton2d_left_hand_end_supported,skeleton2d_right_hand_end_x,skeleton2d_right_hand_end_y,skeleton2d

In [11]:
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))


,dataset_type,scene_id,scene_start,scene_end,frame_nr_global,frame_nr_local,img_id,bb_center_x,bb_center_y,bb_width,bb_height,bb_score,env_position_x,env_position_y,env_position_z,body_orientation_theta,body_orientation_phi,body_orientation_score,head_orientation_theta,head_orientation_phi,head_orientation_score,skeleton2d_nose_x,skeleton2d_nose_y,skeleton2d_nose_score,skeleton2d_left_eye_x,skeleton2d_left_eye_y,skeleton2d_left_eye_score,skeleton2d_right_eye_x,skeleton2d_right_eye_y,skeleton2d_right_eye_score,skeleton2d_left_ear_x,skeleton2d_left_ear_y,skeleton2d_left_ear_score,skeleton2d_right_ear_x,skeleton2d_right_ear_y,skeleton2d_right_ear_score,skeleton2d_left_shoulder_x,skeleton2d_left_shoulder_y,skeleton2d_left_shoulder_score,skeleton2d_right_shoulder_x,skeleton2d_right_shoulder_y,skeleton2d_right_shoulder_score,skeleton2d_left_elbow_x,skeleton2d_left_elbow_y,skeleton2d_left_elbow_score,skeleton2d_right_elbow_x,skeleton2d_right_elbow_y,skeleton2d_right_elbow_score,skeleton2d_left_wrist_x,skeleton2d_left_wrist_y,skeleton2d_left_wrist_score,skeleton2d_right_wrist_x,skeleton2d_right_wrist_y,skeleton2d_right_wrist_score,skeleton2d_left_hip_x,skeleton2d_left_hip_y,skeleton2d_left_hip_score,skeleton2d_right_hip_x,skeleton2d_right_hip_y,skeleton2d_right_hip_score,skeleton2d_left_knee_x,skeleton2d_left_knee_y,skeleton2d_left_knee_score,skeleton2d_right_knee_x,skeleton2d_right_knee_y,skeleton2d_right_knee_score,skeleton2d_left_ankle_x,skeleton2d_left_ankle_y,skeleton2d_left_ankle_score,skeleton2d_right_ankle_x,skeleton2d_right_ankle_y,skeleton2d_right_ankle_score,skeleton2d_hip_center_x,skeleton2d_hip_center_y,skeleton2d_hip_center_score,skeleton2d_spine_center_x,skeleton2d_spine_center_y,skeleton2d_spine_center_score,skeleton2d_neck_x,skeleton2d_neck_y,skeleton2d_neck_score,skeleton2d_head_lower_x,skeleton2d_head_lower_y,skeleton2d_head_lower_score,skeleton2d_head_upper_x,skeleton2d_head_upper_y,skeleton2d_head_upper_score,skeleton2d_left_foot_end_x,skeleton2d_left_foot_end_y,skeleton2d_left_foot_end_score,skeleton2d_right_foot_end_x,skeleton2d_right_foot_end_y,skeleton2d_right_foot_end_score,skeleton2d_left_hand_end_x,skeleton2d_left_hand_end_y,skeleton2d_left_hand_end_score,skeleton2d_right_hand_end_x,skeleton2d_right_hand_end_y,skeleton2d_right_hand_end_score,skeleton3d_nose_x,skeleton3d_nose_y,skeleton3d_nose_z,skeleton3d_nose_score,skeleton3d_left_eye_x,skeleton3d_left_eye_y,skeleton3d_left_eye_z,skeleton3d_left_eye_score,skeleton3d_right_eye_x,skeleton3d_right_eye_y,skeleton3d_right_eye_z,skeleton3d_right_eye_score,skeleton3d_left_ear_x,skeleton3d_left_ear_y,skeleton3d_left_ear_z,skeleton3d_left_ear_score,skeleton3d_right_ear_x,skeleton3d_right_ear_y,skeleton3d_right_ear_z,skeleton3d_right_ear_score,skeleton3d_left_shoulder_x,skeleton3d_left_shoulder_y,skeleton3d_left_shoulder_z,skeleton3d_left_shoulder_score,skeleton3d_right_shoulder_x,skeleton3d_right_shoulder_y,skeleton3d_right_shoulder_z,skeleton3d_right_shoulder_score,skeleton3d_left_elbow_x,skeleton3d_left_elbow_y,skeleton3d_left_elbow_z,skeleton3d_left_elbow_score,skeleton3d_right_elbow_x,skeleton3d_right_elbow_y,skeleton3d_right_elbow_z,skeleton3d_right_elbow_score,skeleton3d_left_wrist_x,skeleton3d_left_wrist_y,skeleton3d_left_wrist_z,skeleton3d_left_wrist_score,skeleton3d_right_wrist_x,skeleton3d_right_wrist_y,skeleton3d_right_wrist_z,skeleton3d_right_wrist_score,skeleton3d_left_hip_x,skeleton3d_left_hip_y,skeleton3d_left_hip_z,skeleton3d_left_hip_score,skeleton3d_right_hip_x,skeleton3d_right_hip_y,skeleton3d_right_hip_z,skeleton3d_right_hip_score,skeleton3d_left_knee_x,skeleton3d_left_knee_y,skeleton3d_left_knee_z,skeleton3d_left_knee_score,skeleton3d_right_knee_x,skeleton3d_right_knee_y,skeleton3d_right_knee_z,skeleton3d_right_knee_score,skeleton3d_left_ankle_x,skeleton3d_left_ankle_y,skeleton3d_left_ankle_z,skeleton3d_left_ankle_score,skeleton3d_right_ankle_x,skeleton3d_right_ankle_y,skeleton3d_right_ankle_z,skeleton3d_right_ankle_score,skeleton3d_hip_center_

In [12]:
df_description = df.describe()
num_3d_models = 1
num_animations = 1
print("General Report")
print(f"Number of frames: {len(df)}")
print(f"FPS: 30")
print(f"Resolution: 1920x1080")
print(f"Number of 3D models: {num_3d_models}")
print(f"Number of animations {num_animations}")
print("Data & Mean & Std & Min & Max \\\\")
print(f"Skeleton 2D Diameter & ${df_description['skeleton2d_size']['mean']:.2f}px$ & ${df_description['skeleton2d_size']['std']:.2f}px$ & ${df_description['skeleton2d_size']['min']:.2f}px$ & ${df_description['skeleton2d_size']['max']:.2f}px$ \\\\")
print(f"Skeleton 3D Diameter & ${df_description['skeleton3d_size']['mean']:.2f}mm$ & ${df_description['skeleton3d_size']['std']:.2f}mm$ & ${df_description['skeleton3d_size']['min']:.2f}mm$ & ${df_description['skeleton3d_size']['max']:.2f}mm$ \\\\")
print(f"Camera Distance (XZ) & ${df_description['distance_xz']['mean']:.2f}mm$ & ${df_description['distance_xz']['std']:.2f}mm$ & ${df_description['distance_xz']['min']:.2f}mm$ & ${df_description['distance_xz']['max']:.2f}mm$ \\\\")

General Report
Number of frames: 1362184
FPS: 30
Resolution: 1920x1080
Number of 3D models: 1
Number of animations 1
Data & Mean & Std & Min & Max \\
Skeleton 2D Diameter & $214.08px$ & $196.61px$ & $9.32px$ & $2201.98px$ \\
Skeleton 3D Diameter & $1855.14mm$ & $230.20mm$ & $78.03mm$ & $2696.13mm$ \\
Camera Distance (XZ) & $16148.65mm$ & $5481.39mm$ & $1640.93mm$ & $76657.67mm$ \\


In [13]:
action_count = {}
for actions in df['actions']:
    for action in actions:
        if action not in action_count:
            action_count[action] = 0
        action_count[action] += 1

# TODO: Actions 
# Which Actions contained
# How many frames per action

In [14]:
print(action_count)
for action_val, count in action_count.items():
    print(f"{ACTION(action_val).name}: {count}")

{1: 859351, 23: 149938, 24: 44036, 25: 50260, 26: 43685, 0: 71977, 9: 10619, 2: 51745, 27: 14428, 21: 9381, 32: 17935, 28: 5848, 29: 14567, 30: 8845, 31: 7871, 19: 45248}
WALK: 859351
LOOK_FOR_TRAFFIC: 149938
HITCHHIKE: 44036
TURN_AROUND: 50260
WORK: 43685
IDLE: 71977
KICK_BALL: 10619
WAVE: 51745
ARGUE: 14428
THROW: 9381
FIGHT: 17935
STUMBLE: 5848
OPEN_DOOR: 14567
FALL: 8845
STAND_UP: 7871
STAND: 45248


In [15]:
fig = plt.figure()
body_phi_deg = df['body_orientation_phi'] * (180 / math.pi)
body_phi_deg.hist(bins=50)
xmin, xmax, ymin, ymax = plt.axis()
tikzplotlib.save("/home/dennis/Downloads/conti_01_body_orientation_phi.tex", extra_axis_parameters={
  'width=0.7\\textwidth',
  'font=\\footnotesize',
  'title={Distribution of body $\\theta$ orientations}',
  'xlabel={degrees ($°$)}',
  'ylabel={Number of samples}',
  'enlarge x limits=0.001',
  'enlarge y limits=0.001',
  'xmin=0',
  'xmax=360',
})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
fig = plt.figure()
head_phi_deg = df['head_orientation_phi'] * (180 / math.pi)
head_phi_deg.hist(bins=50)
xmin, xmax, ymin, ymax = plt.axis()
tikzplotlib.save("/home/dennis/Downloads/conti_01_head_orientation_phi.tex", extra_axis_parameters={
  'width=0.7\\textwidth',
  'font=\\footnotesize',
  'title={Distribution of body $\\theta$ orientations}',
  'xlabel={degrees ($°$)}',
  'ylabel={Number of samples}',
  'enlarge x limits=0.001',
  'enlarge y limits=0.001',
  'xmin=0',
  'xmax=360',
})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
fig = plt.figure()
df['skeleton2d_width'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [18]:
fig = plt.figure()
df['skeleton2d_height'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [19]:
fig = plt.figure()
df['skeleton2d_size'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [20]:
fig = plt.figure()
df['skeleton3d_width'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [21]:
fig = plt.figure()
df['skeleton3d_height'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [22]:
fig = plt.figure()
df['skeleton3d_depth'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [23]:
fig = plt.figure()
df['skeleton3d_size'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [24]:
fig = plt.figure()
df['body_orientation_phi'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [25]:
fig = plt.figure()
df['head_orientation_theta'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [26]:
fig = plt.figure()
df['head_orientation_phi'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [17]:
df_description = df_train.describe()
num_3d_models = 1
num_animations = 1
print("General Report (TRAIN)")
print(f"Number of frames: {len(df_train)}")
print(f"FPS: 30")
print(f"Resolution: 1920x1080")
print(f"Number of 3D models: {num_3d_models}")
print(f"Number of animations {num_animations}")
print("Data & Mean & Std & Min & Max \\\\")
print(f"Skeleton 2D Diameter & ${df_description['skeleton2d_size']['mean']:.2f}px$ & ${df_description['skeleton2d_size']['std']:.2f}px$ & ${df_description['skeleton2d_size']['min']:.2f}px$ & ${df_description['skeleton2d_size']['max']:.2f}px$ \\\\")
print(f"Skeleton 3D Diameter & ${df_description['skeleton3d_size']['mean']:.2f}mm$ & ${df_description['skeleton3d_size']['std']:.2f}mm$ & ${df_description['skeleton3d_size']['min']:.2f}mm$ & ${df_description['skeleton3d_size']['max']:.2f}mm$ \\\\")
print(f"Camera Distance (XZ) & ${df_description['distance_xz']['mean']:.2f}mm$ & ${df_description['distance_xz']['std']:.2f}mm$ & ${df_description['distance_xz']['min']:.2f}mm$ & ${df_description['distance_xz']['max']:.2f}mm$ \\\\")

General Report (TRAIN)
Number of frames: 1233608
FPS: 30
Resolution: 1920x1080
Number of 3D models: 1
Number of animations 1
Data & Mean & Std & Min & Max \\
Skeleton 2D Diameter & $214.55px$ & $197.86px$ & $9.32px$ & $2201.98px$ \\
Skeleton 3D Diameter & $1854.28mm$ & $231.48mm$ & $110.12mm$ & $2696.13mm$ \\
Camera Distance (XZ) & $16134.09mm$ & $5498.43mm$ & $1640.93mm$ & $76657.67mm$ \\


In [18]:
df_description = df_val.describe()
num_3d_models = 1
num_animations = 1
print("General Report (VAL)")
print(f"Number of frames: {len(df_val)}")
print(f"FPS: 30")
print(f"Resolution: 1920x1080")
print(f"Number of 3D models: {num_3d_models}")
print(f"Number of animations {num_animations}")
print("Data & Mean & Std & Min & Max \\\\")
print(f"Skeleton 2D Diameter & ${df_description['skeleton2d_size']['mean']:.2f}px$ & ${df_description['skeleton2d_size']['std']:.2f}px$ & ${df_description['skeleton2d_size']['min']:.2f}px$ & ${df_description['skeleton2d_size']['max']:.2f}px$ \\\\")
print(f"Skeleton 3D Diameter & ${df_description['skeleton3d_size']['mean']:.2f}mm$ & ${df_description['skeleton3d_size']['std']:.2f}mm$ & ${df_description['skeleton3d_size']['min']:.2f}mm$ & ${df_description['skeleton3d_size']['max']:.2f}mm$ \\\\")
print(f"Camera Distance (XZ) & ${df_description['distance_xz']['mean']:.2f}mm$ & ${df_description['distance_xz']['std']:.2f}mm$ & ${df_description['distance_xz']['min']:.2f}mm$ & ${df_description['distance_xz']['max']:.2f}mm$ \\\\")

General Report (VAL)
Number of frames: 128576
FPS: 30
Resolution: 1920x1080
Number of 3D models: 1
Number of animations 1
Data & Mean & Std & Min & Max \\
Skeleton 2D Diameter & $209.55px$ & $184.10px$ & $43.32px$ & $2195.76px$ \\
Skeleton 3D Diameter & $1863.45mm$ & $217.43mm$ & $78.03mm$ & $2450.63mm$ \\
Camera Distance (XZ) & $16288.31mm$ & $5313.15mm$ & $1660.82mm$ & $50025.67mm$ \\


In [18]:
def circular_hist(ax, x, bins=16, density=True, offset=0, gaps=True):
    """
    Produce a circular histogram of angles on ax.

    Parameters
    ----------
    ax : matplotlib.axes._subplots.PolarAxesSubplot
        axis instance created with subplot_kw=dict(projection='polar').

    x : array
        Angles to plot, expected in units of radians.

    bins : int, optional
        Defines the number of equal-width bins in the range. The default is 16.

    density : bool, optional
        If True plot frequency proportional to area. If False plot frequency
        proportional to radius. The default is True.

    offset : float, optional
        Sets the offset for the location of the 0 direction in units of
        radians. The default is 0.

    gaps : bool, optional
        Whether to allow gaps between bins. When gaps = False the bins are
        forced to partition the entire [-pi, pi] range. The default is True.

    Returns
    -------
    n : array or list of arrays
        The number of values in each bin.

    bins : array
        The edges of the bins.

    patches : `.BarContainer` or list of a single `.Polygon`
        Container of individual artists used to create the histogram
        or list of such containers if there are multiple input datasets.
    """
    # Wrap angles to [-pi, pi)
    x = (x+np.pi) % (2*np.pi) - np.pi

    # Force bins to partition entire circle
    if not gaps:
        bins = np.linspace(-np.pi, np.pi, num=bins+1)

    # Bin data and record counts
    n, bins = np.histogram(x, bins=bins)

    # Compute width of each bin
    widths = np.diff(bins)

    # By default plot frequency proportional to area
    if density:
        # Area to assign each bin
        area = n / x.size
        # Calculate corresponding bin radius
        radius = (area/np.pi) ** .5
    # Otherwise plot frequency proportional to radius
    else:
        radius = n

    # Plot data on ax
    patches = ax.bar(bins[:-1], radius, zorder=1, align='edge', width=widths,
                     edgecolor='C0', color='orange', fill=True, linewidth=1)

    # Set the direction of the zero angle
    #ax.set_theta_offset(offset)

    # Remove ylabels for area plots (they are mostly obstructive)
    if density:
        ax.set_yticks([])

    return n, bins, patches

In [21]:
# Construct figure and axis to plot on
# fig, ax = plt.subplots(1, 2)
fig = plt.figure()
# ax = fig.add_subplot(projection='polar')
ax = fig.add_subplot()
body_phi_deg = df_train['body_orientation_phi']  * (180 / math.pi)
# body_phi_deg.hist(bins=360)
# fig, ax = plt.subplots(1, 2, subplot_kw=dict(projection='polar'))
# Visualise by area of bins
# circular_hist(ax[0], body_phi_deg)
# Visualise by radius of bins
circular_hist(ax, body_phi_deg, bins=16, offset=0, density=False)
xmin, xmax, ymin, ymax = plt.axis()
tikzplotlib.save("/home/dennis/Downloads/c01_train_body_orientation_phi.tex", extra_axis_parameters={
  'width=0.5\\textwidth',
  'title={Distribution of body $\\theta$ orientations}',
  'xlabel={degrees [$°$]}',
  'ylabel={Number of samples}',
#  'enlarge x limits=0.001',
#  'enlarge y limits=0.001',
#  'xmin=0',
#  'xmax=360',
})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
body_phi_deg = df['body_orientation_phi'] * (180 / math.pi)
max_val = 0
for i in range (0, 370, 10):
  elements = body_phi_deg[body_phi_deg.between(i, i+10, "both")]
  num_els = len(elements)
  max_val = max_val if max_val > num_els else num_els
  print(f"{i},{num_els}")
print(max_val)

0,47463
10,22051
20,15087
30,12702
40,13588
50,14593
60,15993
70,31187
80,166247
90,171807
100,38010
110,20763
120,16416
130,14996
140,13227
150,15707
160,26258
170,61853
180,52793
190,24259
200,15911
210,13133
220,13933
230,14715
240,15902
250,28999
260,147567
270,153217
280,33400
290,18407
300,14249
310,12469
320,10762
330,12762
340,21805
350,53815
360,0


In [10]:
body_phi_deg = df_val['body_orientation_phi'] * (180 / math.pi)
max_val = 0
for i in range (0, 370, 10):
  elements = body_phi_deg[body_phi_deg.between(i, i+10, "both")]
  num_els = len(elements)
  max_val = max_val if max_val > num_els else num_els
  print(f"{i},{num_els}")

0,5998
10,1715
20,1282
30,1006
40,824
50,680
60,805
70,2416
80,16424
90,16656
100,2514
110,1739
120,1364
130,1601
140,1088
150,1300
160,2137
170,8821
180,6359
190,1876
200,1360
210,1036
220,905
230,707
240,881
250,2516
260,14197
270,14639
280,2205
290,1547
300,1110
310,1198
320,836
330,988
340,1573
350,7673
360,0


In [23]:
fig = plt.figure()
body_phi_deg = df_train['body_orientation_phi'] * (180 / math.pi)
body_phi_deg.hist(bins=50)
xmin, xmax, ymin, ymax = plt.axis()
tikzplotlib.save("/home/dennis/Downloads/c01_train_body_orientation_phi.tex", extra_axis_parameters={
  'width=0.5\\textwidth',
  'title={Distribution of body $\\theta$ orientations}',
  'xlabel={degrees ($°$)}',
  'ylabel={Number of samples}',
  'enlarge x limits=0.001',
  'enlarge y limits=0.001',
  'xmin=0',
  'xmax=360',
})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fig = plt.figure()
head_phi_deg = df_train['head_orientation_phi'] * (180 / math.pi)
head_phi_deg.hist(bins=50)
xmin, xmax, ymin, ymax = plt.axis()
tikzplotlib.save("/home/dennis/Downloads/c01_train_head_orientation_phi.tex", extra_axis_parameters={
  'width=0.5\\textwidth',
  'title={Distribution of body $\\theta$ orientations}',
  'xlabel={degrees ($°$)}',
  'ylabel={Number of samples}',
  'enlarge x limits=0.001',
  'enlarge y limits=0.001',
  'xmin=0',
  'xmax=360',
})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fig = plt.figure()
body_phi_deg = df_val['body_orientation_phi'] * (180 / math.pi)
body_phi_deg.hist(bins=50)
xmin, xmax, ymin, ymax = plt.axis()
tikzplotlib.save("/home/dennis/Downloads/c01_val_body_orientation_phi.tex", extra_axis_parameters={
  'width=0.5\\textwidth',
  'title={Distribution of body $\\theta$ orientations}',
  'xlabel={degrees ($°$)}',
  'ylabel={Number of samples}',
  'enlarge x limits=0.001',
  'enlarge y limits=0.001',
  'xmin=0',
  'xmax=360',
})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
fig = plt.figure()
head_phi_deg = df_val['head_orientation_phi'] * (180 / math.pi)
head_phi_deg.hist(bins=50)
xmin, xmax, ymin, ymax = plt.axis()
tikzplotlib.save("/home/dennis/Downloads/c01_val_head_orientation_phi.tex", extra_axis_parameters={
  'width=0.5\\textwidth',
  'title={Distribution of body $\\theta$ orientations}',
  'xlabel={degrees ($°$)}',
  'ylabel={Number of samples}',
  'enlarge x limits=0.001',
  'enlarge y limits=0.001',
  'xmin=0',
  'xmax=360',
})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
print("3D Result Evaluation")

In [10]:
df_results = pd.read_pickle("data/datasets/Conti01/results/C01_pred_df_experiment_pedrec_p2d3d_c_o_h36m_sim_mebow_0.pkl")
dataset_df_val_path = os.path.join(dataset_root, "rt_conti_01_val.pkl")
df = pd.read_pickle(dataset_df_val_path)

print(len(df_results))
print(len(df_val))

119257
119257


In [ ]:
env_position_xs = [col for col in df if col == "env_position_x"] 
env_position_ys = [col for col in df if col == "env_position_y"] 
env_position_zs = [col for col in df if col == "env_position_z"] 

df["distance_xz"] = np.sqrt(np.abs(df[env_position_xs].sum(axis=1)**2) + np.abs(df[env_position_zs].sum(axis=1)**2))
df_results["distance_xz"] = df["distance_xz"]

In [ ]:
pck_results = get_2d_pose_pck_results(skeleton_2d_gt, skeleton_2d_pred)